<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# <h1 align="center" id="heading">Sentiment Analysis of Reddit Data using Reddit API</h1>

In this live coding session, we leverage the Python Reddit API Wrapper (`PRAW`) to retrieve data from subreddits on [Reddit](https://www.reddit.com), and perform sentiment analysis using [`pipelines`](https://huggingface.co/docs/transformers/main_classes/pipelines) from [HuggingFace ( 🤗 the GitHub of Machine Learning )](https://techcrunch.com/2022/05/09/hugging-face-reaches-2-billion-valuation-to-build-the-github-of-machine-learning/), powered by [transformer](https://arxiv.org/pdf/1706.03762.pdf).

## Objectives

At the end of the session, you will 

- know how to work with APIs
- feel more comfortable navigating thru documentation, even inspecting the source code
- understand what a `pipeline` object is in HuggingFace
- perform sentiment analysis using `pipeline`
- run a python script in command line and get the results

## How to Submit

- At the end of each task, commit* the work into the repository you created before the assignment
- After completing all three tasks, make sure to push the notebook containing all code blocks and output cells to your repository you created before the assignment
- Submit the link to the notebook in Canvas

\***NEVER** commit a notebook displaying errors unless it is instructed otherwise. However, commit often; recall git ABC = **A**lways **B**e **C**ommitting.

## Tasks

### Task I: Instantiate a Reddit API Object

The first task is to instantiate a Reddit API object using [PRAW](https://praw.readthedocs.io/en/stable/), through which you will retrieve data. PRAW is a wrapper for [Reddit API](https://www.reddit.com/dev/api) that makes interacting with the Reddit API easier unless you are already an expert of [`requests`](https://docs.python-requests.org/en/latest/).

#### 1. Install packages

Please ensure you've ran all the cells in the `imports.ipynb`, located [here](https://github.com/FourthBrain/MLE-8/blob/main/assignments/week-3-analyze-sentiment-subreddit/imports.ipynb), to make sure you have all the required packages for today's assignment.

####  2. Create a new app on Reddit 

Create a new app on Reddit and save secret tokens; refer to [post in medium](https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c) for more details.

- Create a Reddit account if you don't have one, log into your account.
- To access the API, we need create an app. Slight updates, on the website, you need to navigate to `preference` > `app`, or click [this link](https://www.reddit.com/prefs/apps) and scroll all the way down. 
- Click to create a new app, fill in the **name**, choose `script`, fill in  **description** and **redirect uri** ( The redirect URI is where the user is sent after they've granted OAuth access to your application (more info [here](https://github.com/reddit-archive/reddit/wiki/OAuth2)) For our purpose, you can enter some random url, e.g., www.google.com; as shown below.


    <img src="https://miro.medium.com/max/700/1*lRBvxpIe8J2nZYJ6ucMgHA.png" width="500"/>
- Jot down `client_id` (left upper corner) and `client_secret` 

    NOTE: CLIENT_ID refers to 'personal use script" and CLIENT_SECRET to secret.
    
    <div>
    <img src="https://miro.medium.com/max/700/1*7cGAKth1PMrEf2sHcQWPoA.png" width="300"/>
    </div>

- Create `secrets_reddit.py` in the same directory with this notebook, fill in `client_id` and `secret_id` obtained from the last step. We will need to import those constants in the next step.
    ```
    REDDIT_API_CLIENT_ID = "client_id"
    REDDIT_API_CLIENT_SECRET = "secret_id"
    REDDIT_API_USER_AGENT = "any string except bot; ex. My User Agent"
    ```
- Add `secrets_reddit.py` to your `.gitignore` file if not already done. NEVER push credentials to a repo, private or public. 

#### 3. Instantiate a `Reddit` object

Now you are ready to create a read-only `Reddit` instance. Refer to [documentation](https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html) when necessary.

In [24]:
import praw
import secrets_reddit

# Create a Reddit object which allows us to interact with the Reddit API
reddit = praw.Reddit(
    client_id=secrets_reddit.REDDIT_API_CLIENT_ID,
    client_secret=secrets_reddit.REDDIT_API_CLIENT_SECRET,
    user_agent=secrets_reddit.REDDIT_API_USER_AGENT
)

In [25]:
print(reddit)

<details>
<summary>Expected output:</summary>   

```<praw.reddit.Reddit object at 0x10f8a0ac0>```
</details>

#### 4. Instantiate a `subreddit` object

Lastly, create a `subreddit` object for your favorite subreddit and inspect the object. The expected outputs you will see are from `r/machinelearning` unless otherwise specified.

In [38]:
subreddit = reddit.subreddit("machinelearning")

What is the display name of the subreddit?

In [39]:
print(subreddit.display_name)

machinelearning


<details>
<summary>Expected output:</summary>   

    machinelearning
</details>

How about its title, is it different from the display name?

In [40]:
print(subreddit.title)

Machine Learning


<details>
<summary>Expected output:</summary>   

    Machine Learning
</details>

Print out the description of the subreddit:

In [42]:
print(subreddit.description[:400])

**[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
--------
+[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
--------
+[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
--------
+[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q


<details>
<summary>Expected output:</summary>

    **[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
    --------
    +[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
    --------
    +[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
    --------
    +[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
    --------
    +[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict
</details>

### Task II: Parse comments

#### 1. Top Posts of All Time

Find titles of top 10 posts of **all time** from your favorite subreddit. Refer to [Obtain Submission Instances from a Subreddit Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html)) if necessary. Verify if the titles match what you read on Reddit.

In [43]:
# try run this line, what do you see? press q once you are done
?subreddit.top 

In [44]:
for i in subreddit.top(limit=10):
    print(i.title)

[Project] From books to presentations in 10s with AR + ML
[D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
[R] First Order Motion Model applied to animate paintings
[N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
[D] This AI reveals how much time politicians stare at their phone at work
[D] Types of Machine Learning Papers
[D] The machine learning community has a toxicity problem
I made a robot that punishes me if it detects that if I am procrastinating on my assignments [P]
[Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
[P] Using oil portraits and First Order Model to bring the paintings back to life


<details> <summary>Expected output:</summary>

    [Project] From books to presentations in 10s with AR + ML
    [D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
    [R] First Order Motion Model applied to animate paintings
    [N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
    [D] This AI reveals how much time politicians stare at their phone at work
    [D] Types of Machine Learning Papers
    [D] The machine learning community has a toxicity problem
    [Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
    [P] Using oil portraits and First Order Model to bring the paintings back to life
    [D] Convolution Neural Network Visualization - Made with Unity 3D and lots of Code / source - stefsietz (IG)    
</details>

#### 2. Top 10 Posts of This Week

What are the titles of the top 10 posts of **this week** from your favorite subreddit?

In [45]:
for i in subreddit.top(limit=10, time_filter="week"):
    print(i.title)

[P] Building a App for Stable Diffusion: Text to Image generation in Python
[R] Sketch2Pose — estimating a 3D character pose from a bitmap sketch
[D] StableDiffusion v1.4 is entirely public. What do you think about Stability.ai ?
[D] How to Run Stable Diffusion (Locally and in Colab)
[D] In 2010, did people expect things like DALLE and AlphaFold to be only 10/13 years away?
[D] ML for Good
[N] John Carmack raises $20M from various investors to start Keen Technologies, an AGI Company.
[D][N]"Mudge learned that Twitter had never acquired proper legal rights to training material used to build Twitter's key Machine Learning models. The Machine Learning models at issue were some of the core models running the company's most basic products, like which Tweets to show each user."
[R] Musika! Fast Infinite Waveform Music Generation + Gradio Web Demo
[D] Has any non-PhD here published an ML paper?


<details><summary>Expected output:</summary>

    [N] Ian Goodfellow, Apple’s director of machine learning, is leaving the company due to its return to work policy. In a note to staff, he said “I believe strongly that more flexibility would have been the best policy for my team.” He was likely the company’s most cited ML expert.
    [R][P] Thin-Plate Spline Motion Model for Image Animation + Gradio Web Demo
    [P] I’ve been trying to understand the limits of some of the available machine learning models out there. Built an app that lets you try a mix of CLIP from Open AI + Apple’s version of MobileNet, and more directly on your phone's camera roll.
    [R] Meta is releasing a 175B parameter language model
    [N] Hugging Face raised $100M at $2B to double down on community, open-source & ethics
    [P] T-SNE to view and order your Spotify tracks
    [D] : HELP Finding a Book - A book written for Google Engineers about foundational Math to support ML
    [R] Scaled up CLIP-like model (~2B) shows 86% Zero-shot on Imagenet
    [D] Do you use NLTK or Spacy for text preprocessing?
    [D] Democratizing Diffusion Models - LDMs: High-Resolution Image Synthesis with Latent Diffusion Models, a 5-minute paper summary by Casual GAN Papers
</details>

💽❓ Data Question:

Check out what other attributes the `praw.models.Submission` class has in the [docs](https://praw.readthedocs.io/en/stable/code_overview/models/submission.html). 

1. After having a chance to look through the docs, is there any other information that you might want to extract? How might this additional data help you?

Write a sample piece of code below extracting three additional pieces of information from the submission below.

In [46]:
import datetime

for i in subreddit.top(time_filter="all", limit=10):
    print("Created_Utc: {}, Number of comments: {}, Author: {}, Score: {}".format(datetime.datetime.utcfromtimestamp(i.created_utc)
, i.num_comments, i.author, i.score))

Created_Utc: 2020-05-10 13:19:54, Number of comments: 183, Author: cyrildiagne, Score: 7392
Created_Utc: 2021-01-10 10:30:36, Number of comments: 128, Author: TheInsaneApp, Score: 5637
Created_Utc: 2020-04-25 04:27:23, Number of comments: 110, Author: programmerChilli, Score: 4657
Created_Utc: 2021-02-28 15:12:28, Number of comments: 230, Author: TheInsaneApp, Score: 4576
Created_Utc: 2021-07-11 04:18:59, Number of comments: 228, Author: TheInsaneApp, Score: 4322
Created_Utc: 2021-05-01 09:32:20, Number of comments: 218, Author: TheInsaneApp, Score: 4201
Created_Utc: 2020-06-30 20:06:19, Number of comments: 569, Author: yusuf-bengio, Score: 3696
Created_Utc: 2022-06-26 05:52:23, Number of comments: 158, Author: _ayushp_, Score: 3551
Created_Utc: 2021-03-14 21:46:35, Number of comments: 172, Author: mencil47, Score: 3491
Created_Utc: 2020-09-27 10:31:57, Number of comments: 114, Author: Enguzelharf, Score: 3392


💽❓ Data Question:

2. Is there any information available that might be a concern when it comes to Ethical Data?

In [23]:
# [D][N]"Mudge learned that Twitter had never acquired proper legal rights to training
# material used to build Twitter's key Machine Learning models. The Machine Learning models at
# issue were some of the core models running the company's most basic products, like which Tweets to
# show each user."

#### 3. Comment Code

Add comments to the code block below to describe what each line of the code does (Refer to [Obtain Comment Instances Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html) when necessary). The code is adapted from [this tutorial](https://praw.readthedocs.io/en/stable/tutorials/comments.html)

The purpose is 
1. to understand what the code is doing 
2. start to comment your code whenever it is not self-explantory if you have not (others will thank you, YOU will thank you later 😊) 

In [47]:
%%time
from praw.models import MoreComments

# Instantiating an empty list;
top_comments = []

# Looping through the subreddit.top 10 elements;
for submission in subreddit.top(limit=10):
    # top_level_comment in each submission element above
    for top_level_comment in submission.comments:
        # Checking if top_level_comment is part of the MoreComments class; if it is true continue,
        # else breaks
        if isinstance(top_level_comment, MoreComments):
            continue
        # On the top_comments list append the top_level_comment.body
        top_comments.append(top_level_comment.body)

CPU times: user 372 ms, sys: 16.6 ms, total: 389 ms
Wall time: 13.3 s


#### 4. Inspect Comments

How many comments did you extract from the last step? Examine a few comments. 

In [48]:
# YOUR CODE HERE
# the answer may vary 693 for r/machinelearning
len(top_comments)

741

In [50]:
import random

[random.choice(top_comments) for i in range(5)]

['Potter Tech',
 'This is probably the coolest thing I have seen in a long while.  Great fucking work!',
 'does it pass the Turing test tho lol',
 'Does that mean they are on Reddit ?',
 'Source: https://driesdepoorter.be/theflemishscrollers/']

<details> <summary>Some of the comments from `r/machinelearning` subreddit are:</summary>

    ['Awesome visualisation',
    'Similar to a stack or connected neurons.',
    'Will this Turing pass the Turing Test?']
</details>

In [51]:
['Potter Tech',
 'This is probably the coolest thing I have seen in a long while.  Great fucking work!',
 'does it pass the Turing test tho lol',
 'Does that mean they are on Reddit ?',
 'Source: https://driesdepoorter.be/theflemishscrollers/']

['Potter Tech',
 'This is probably the coolest thing I have seen in a long while.  Great fucking work!',
 'does it pass the Turing test tho lol',
 'Does that mean they are on Reddit ?',
 'Source: https://driesdepoorter.be/theflemishscrollers/']

💽❓ Data Question:

3. After having a chance to review a few samples of 5 comments from the subreddit, what can you say about the data? 

HINT: Think about the "cleanliness" of the data, the content of the data, think about what you're trying to do - how does this data line up with your goal?

In [52]:
# Data is really dirty, lot's of big strings hard to get the information out of it...

#### 5. Extract Top Level Comment from Subreddit `TSLA`.

Write your code to extract top level comments from the top 10 topics of a time period, e.g., year, from subreddit `TSLA` and store them in a list `top_comments_tsla`.  

In [54]:
# YOUR CODE HERE
subreddit2 = reddit.subreddit("TSLA")

for i in subreddit2.top(limit=10):
    print(i.title)

I TOLD YOU SO
Up Vote if you would like to see a 1for 5 stock split?
I cry every time 😢
Raise your hands if you stayed long and didn’t de-risk on Tesla because of your research and faith in the actual company.
LOL?
Veteran TSLA hodler. Don't fall for all the bad news. Been hodling since 2015 and I am still proud of my 10k turned 200k and now 120k. F#$* you shorts. Don't forget the last couple years!!! In the end I win again, again, and again. Have fun today. My plan will beat yours!!! Paper hands!
Vote up for a 10 for 1 stock split (at 1500/share before split)!
Genesis 7:1 💎🤲
$TSLA will bounce back like we’ve never been before. Not selling a single share. I just wish I could buy more at these discounted prices. 💎🙌🏻🚀
HODLING 243 shrs of TSLA and have $0 buying power


In [55]:
%%time
from praw.models import MoreComments

# Instantiating an empty list;
top_comments_tsla = []

# Looping through the subreddit.top 10 elements;
for submission in subreddit2.top(limit=10):
    # top_level_comment in each submission element above
    for top_level_comment in submission.comments:
        # Checking if top_level_comment is part of the MoreComments class; if it is true continue,
        # else breaks
        if isinstance(top_level_comment, MoreComments):
            continue
        # On the top_comments list append the top_level_comment.body
        top_comments_tsla.append(top_level_comment.body)

CPU times: user 94.4 ms, sys: 0 ns, total: 94.4 ms
Wall time: 4.09 s


In [56]:
len(top_comments_tsla) # Expected: 174 for r/machinelearning

170

In [57]:
[random.choice(top_comments_tsla) for i in range(5)]

["Isn't it solely up to Elon?",
 'I laughed heartily :)\n\nI sold some of my shares at loss and bought $590 call options. I hope it is near the bottom. Otherwise, I am screwed too.',
 'Nope I wussed out with the economy and Russia. And to top it off I bought a lot near the peak- $1052 and sold it off well under that. \n\nFortunately I lost $50 only. I bought/sold more as it made its descent into hell and back. \n\nUnfortunately I should have held when it was at below $800. \n\nI’ll buy it. But not at $1000.  I simply cannot imagine this going to $2000 with all that’s going outside of Tesla’s control. \n\nI’ll look at $900, maybe dip in at $800 and love it at $700.',
 '😂😂😂 hold it! Be a macho and hold till the end 😂😂😂',
 'How high is it projected to go? Just found out I have quite a few shares still from when I bought years ago. Sold a few of them back then and somehow forgot I had others!']

<details>
<summary>Some of the comments from `r/TSLA` subreddit:</summary>

    ['I bought puts',
    '100%',
    'Yes. And I’m bag holding 1200 calls for Friday and am close to throwing myself out the window']
</details>

💽❓ Data Question:

4. Now that you've had a chance to review another subreddits comments, do you see any differences in the kinds of comments either subreddit has - and how might this relate to bias?

In [58]:
# I think the data is still messy and but definitely creating bias in the community

### Task III: Sentiment Analysis

Let us analyze the sentiment of comments scraped from `r/TSLA` using a pre-trained HuggingFace model to make the inference. Take a [Quick tour](https://huggingface.co/docs/transformers/quicktour). 

#### 1. Import `pipeline`

In [59]:
from transformers import pipeline

#### 2. Create a Pipeline to Perform Task "sentiment-analysis"

In [72]:
sentiment_model = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


#### 3. Get one comment from list `top_comments_tsla` from Task II - 5.

In [73]:
comment = random.choice(top_comments_tsla)

In [74]:
comment

"\nRight now, it's actually a great opportunity to buy TSLA at such a huge discount. When it's back over 800, we'll already have locked in a 35% gain! 😎👍"

The example comment is: `'Bury Burry!!!!!'`. Print out what you get. For reproducibility, use the same comment in the next step; consider setting a seed.

#### 4. Make Inference!

In [79]:
sentiment = sentiment_model(top_comments_tsla)

In [80]:
sentiment

[{'label': 'NEGATIVE', 'score': 0.9937398433685303},
 {'label': 'NEGATIVE', 'score': 0.9993293285369873},
 {'label': 'NEGATIVE', 'score': 0.9959704875946045},
 {'label': 'NEGATIVE', 'score': 0.9924914240837097},
 {'label': 'NEGATIVE', 'score': 0.9972865581512451},
 {'label': 'POSITIVE', 'score': 0.9799702167510986},
 {'label': 'NEGATIVE', 'score': 0.9992691874504089},
 {'label': 'POSITIVE', 'score': 0.6894425749778748},
 {'label': 'POSITIVE', 'score': 0.9486647844314575},
 {'label': 'POSITIVE', 'score': 0.9979830980300903},
 {'label': 'POSITIVE', 'score': 0.9963148236274719},
 {'label': 'NEGATIVE', 'score': 0.9895860552787781},
 {'label': 'POSITIVE', 'score': 0.5503337979316711},
 {'label': 'POSITIVE', 'score': 0.9739510416984558},
 {'label': 'NEGATIVE', 'score': 0.9952527284622192},
 {'label': 'NEGATIVE', 'score': 0.9689355492591858},
 {'label': 'POSITIVE', 'score': 0.9997166991233826},
 {'label': 'NEGATIVE', 'score': 0.9991776347160339},
 {'label': 'NEGATIVE', 'score': 0.966599225997

What is the type of the output `sentiment`?

```
YOUR ANSWER HERE
```

In [81]:
print(f'The comment: {comment}')
print(f'Predicted Label is {sentiment[0]["label"]} and the score is {sentiment[0]["score"]:.3f}')

The comment: 
Right now, it's actually a great opportunity to buy TSLA at such a huge discount. When it's back over 800, we'll already have locked in a 35% gain! 😎👍
Predicted Label is NEGATIVE and the score is 0.994


For the example comment, the output is:

    The comment: Bury Burry!!!!!
    Predicted Label is NEGATIVE and the score is 0.989

🖥️❓ Model Question:

1. What does the score represent?

In [82]:
# On this exercise this model was not as good; the classification came back wrong

### Task IV: Put All Together

Let's pull all the piece together, create a simple script that does 

- get the subreddit
- get comments from the top posts for given subreddit
- run sentiment analysis 

#### Complete the Script

Once you complete the code, running the following block writes the code into a new Python script and saves it as `top_tlsa_comment_sentiment.py` under the same directory with the notebook. 

In [87]:
%%writefile top_tlsa_comment_sentiment.py

import secrets_reddit
import random

from typing import Dict, List

from praw import Reddit
from praw.models.reddit.subreddit import Subreddit
from praw.models import MoreComments

from transformers import pipeline


def get_subreddit(display_name:str) -> Subreddit:
    """Get subreddit object from display name

    Args:
        display_name (str): [description]

    Returns:
        Subreddit: [description]
    """
    reddit = Reddit(
        client_id=secrets_reddit.REDDIT_API_CLIENT_ID,        
        client_secret=secrets_reddit.REDDIT_API_CLIENT_SECRET,
        user_agent=secrets_reddit.REDDIT_API_USER_AGENT
        )
    
    subreddit = reddit.subreddit(display_name)
    return subreddit

def get_comments(subreddit:Subreddit, limit:int=3) -> List[str]:
    """ Get comments from subreddit

    Args:
        subreddit (Subreddit): [description]
        limit (int, optional): [description]. Defaults to 3.

    Returns:
        List[str]: List of comments
    """
    top_comments = []
    for submission in subreddit.top(limit=limit):
        for top_level_comment in submission.comments:
            if isinstance(top_level_comment, MoreComments):
                continue
            top_comments.append(top_level_comment.body)
    return top_comments

def run_sentiment_analysis(comment:str) -> Dict:
    """Run sentiment analysis on comment using default distilbert model
    
    Args:
        comment (str): [description]
        
    Returns:
        str: Sentiment analysis result
    """
    sentiment_model = pipeline("sentiment-analysis")
    sentiment = sentiment_model(comment)
    return sentiment[0]


if __name__ == '__main__':
    subreddit = get_subreddit("TSLA")
    comments = get_comments(subreddit)
    comment = random.choice(comments)
    sentiment = run_sentiment_analysis(comment)
    
    print(f'The comment: {comment}')
    print(f'Predicted Label is {sentiment["label"]} and the score is {sentiment["score"]:.3f}')

Overwriting top_tlsa_comment_sentiment.py


Run the following block to see the output.

In [88]:
!python top_tlsa_comment_sentiment.py

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
The comment: Which other Crypto do Tesla plan to approve?
Predicted Label is NEGATIVE and the score is 0.997


<details><summary> Expected output:</summary>

    No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
    The comment: When is DOGE flying
    Predicted Label is POSITIVE and the score is 0.689
</details>

💽❓ Data Question:

5. Is the subreddit active? About how many posts or threads per day? How could you find this information?

In [89]:
# Yes, it is; based on the counts of comments in the output comments.

💽❓ Data Question:

6. Does there seem to be a large distribution of posters or a smaller concentration of posters who are very active? What kind of impact might this have on the data?

In [ ]:
# Definitely some posters are more active than others; We can have bias in the data by definition.